In [1]:
import numpy as np
import pandas as pd



# Load Data

In [2]:
full_data = pd.read_csv('dataset/application_record.csv')
status_data = pd.read_csv('dataset/credit_record.csv')

print(full_data.shape)
print(status_data.shape)

(438557, 18)
(1048575, 3)


In [3]:
full_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [4]:
status_data.head()

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


**MONTHS_BALANCE**	Record month	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on

<br>

**STATUS**	Status	0: 1-29 days past due 1: 30-59 days past due 2: 60-89 days overdue 3: 90-119 days overdue 4: 120-149 days overdue 5: Overdue or bad debts, write-offs for more than 150 days C: paid off that month X: No loan for the month

# Create the target variable

In [5]:
month_begin = status_data.groupby(['ID'])['MONTHS_BALANCE'].min().reset_index()
month_begin = month_begin.rename(columns={'MONTHS_BALANCE':'ACCT_AGE'})
month_begin.head()

,ID,ACCT_AGE
0,5001711,-3
1,5001712,-18
2,5001713,-21
3,5001714,-14
4,5001715,-59


In [6]:
full_data_merge_age = full_data.merge(month_begin, on = 'ID', how = 'left')
print(full_data_merge_age.shape)
full_data_merge_age.head()

(438557, 19)


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ACCT_AGE
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-15.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-14.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-29.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-4.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0


In [7]:
status_data.STATUS.unique()

array(['X', '0', 'C', '1', '2', '3', '4', '5'], dtype=object)

In [8]:
# if status is not (X or C,0,1), it is past due
# if x and c is not high risk else yes

status_data['dep_value'] = status_data.STATUS.apply(lambda x: None if x == 'X' or x == 'C' or x == '0' or x == '1'else 'Y')
status_data['dep_value'].unique()

array([None, 'Y'], dtype=object)

In [9]:
target_df = status_data.groupby(['ID']).count().reset_index()
target_df = target_df.rename(columns = {'dep_value':'HIGH_RISK'})
target_df['HIGH_RISK'] = target_df.HIGH_RISK.apply(lambda x: 0 if x == 0 else 1)
target_df.head()


,ID,MONTHS_BALANCE,STATUS,HIGH_RISK
0,5001711,4,4,0
1,5001712,19,19,0
2,5001713,22,22,0
3,5001714,15,15,0
4,5001715,60,60,0


In [10]:
full_data_merge_age_risk = full_data_merge_age.merge(target_df[['ID','HIGH_RISK']], how = 'inner', on = 'ID')
full_data_merge_age_risk.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,ACCT_AGE,HIGH_RISK
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-15.0,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,-14.0,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,-29.0,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-4.0,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,-26.0,0


# EDA

In [11]:
full_data_merge_age_risk.iloc[:,1:].describe()

,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,ACCT_AGE,HIGH_RISK
count,36457.000000,3.645700e+04,36457.000000,36457.000000,36457.0,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000,36457.000000
mean,0.430315,1.866857e+05,-15975.173382,59262.935568,1.0,0.225526,0.294813,0.089722,2.198453,-26.164193,0.016897
std,0.742367,1.017892e+05,4200.549944,137651.334859,0.0,0.417934,0.455965,0.285787,0.911686,16.501854,0.128886
min,0.000000,2.700000e+04,-25152.000000,-15713.000000,1.0,0.000000,0.000000,0.000000,1.000000,-60.000000,0.000000
25%,0.000000,1.215000e+05,-19438.000000,-3153.000000,1.0,0.000000,0.000000,0.000000,2.000000,-39.000000,0.000000
50%,0.000000,1.575000e+05,-15563.000000,-1552.000000,1.0,0.000000,0.000000,0.000000,2.000000,-24.000000,0.000000
75%,1.000000,2.250000e+05,-12462.000000,-408.000000,1.0,0.000000,1.000000,0.000000,3.000000,-12.000000,0.000000
max,19.000000,1.575000e+06,-7489.000000,365243.000000,1.0,1.000000,1.000000,1.000000,20.000000,0.000000,1.000000


## value counts nd frequency of each observation within a feature

In [12]:
def value_cnt_norm(df,feature):
    
    fv_cnt = df[feature].value_counts()
    fv_cnt_norm = df[feature].value_counts(normalize=True)*100
    fv_cnt_df = pd.concat([fv_cnt,fv_cnt_norm], axis = 1)

    fv_cnt_df.columns = [f'{feature}_CNT','FREQUENCY']
    return fv_cnt_df

In [13]:
[value_cnt_norm(full_data_merge_age_risk,i) for i in full_data_merge_age_risk.columns if i != 'ID' ]

[   CODE_GENDER_CNT  FREQUENCY
 F            24430  67.010451
 M            12027  32.989549,
    FLAG_OWN_CAR_CNT  FREQUENCY
 N             22614   62.02924
 Y             13843   37.97076,
    FLAG_OWN_REALTY_CNT  FREQUENCY
 Y                24506  67.218915
 N                11951  32.781085,
     CNT_CHILDREN_CNT  FREQUENCY
 0              25201  69.125271
 1               7492  20.550237
 2               3256   8.931069
 3                419   1.149299
 4                 63   0.172806
 5                 20   0.054859
 14                 3   0.008229
 7                  2   0.005486
 19                 1   0.002743,
           AMT_INCOME_TOTAL_CNT  FREQUENCY
 135000.0                  4309  11.819404
 180000.0                  3097   8.494939
 157500.0                  3089   8.472996
 112500.0                  2956   8.108182
 225000.0                  2926   8.025894
 ...                        ...        ...
 177750.0                     1   0.002743
 594000.0                   

<h1 style="color:darkred">NOTE</h1>

<ul>
    <li> DAYS_BIRTH	Birthday	Count backwards from current day (0), -1 means yesterday</li>
    <li>DAYS_EMPLOYED	Start date of employment	Count backwards from current day(0). If positive, it means the person currently unemployed.</li>
 </ul>

# Bracket AMT income


In [14]:
print('\033[93m')
print('The lowest income: ${}\nThe median income: ${}\nThe highest income: ${}'\
      .format(full_data_merge_age_risk['AMT_INCOME_TOTAL'].min(),\
             full_data_merge_age_risk['AMT_INCOME_TOTAL'].mean(),\
             full_data_merge_age_risk['AMT_INCOME_TOTAL'].max()))


The lowest income: $27000.0
The median income: $186685.73666236937
The highest income: $1575000.0


In [15]:
print('\033[92m')

print('''I decide to bracket the amt income into 4 brackets:\033[93m \n\033[1m1st bracket:\033[0m \033[93m{} records which is the group of customer who earned income from ${} to $100,000\n\033[1m2nd bracket:\033[0m \033[93m{} records which is the group of customer who earned income from $100,000 to $200,000\n\033[1m3rd bracket:\033[0m \033[93m{} records which is the group of customer who earned income from $200,000 to $400,000\n\033[1m4th bracket:\033[0m \033[93m{} records which is the group of customer who earned income more than $400,000'''\
      .format(full_data_merge_age_risk[full_data_merge_age_risk['AMT_INCOME_TOTAL'] < 100000].shape[0],
              full_data_merge_age_risk['AMT_INCOME_TOTAL'].min(),
              full_data_merge_age_risk.query('AMT_INCOME_TOTAL >= 100000 & AMT_INCOME_TOTAL < 200000').shape[0],
              full_data_merge_age_risk.query('AMT_INCOME_TOTAL >= 200000 & AMT_INCOME_TOTAL < 400000').shape[0],
              full_data_merge_age_risk.query('AMT_INCOME_TOTAL >= 400000').shape[0]))


I decide to bracket the amt income into 4 brackets: 
1st bracket: 5086 records which is the group of customer who earned income from $27000.0 to $100,000
2nd bracket: 17897 records which is the group of customer who earned income from $100,000 to $200,000
3rd bracket: 12169 records which is the group of customer who earned income from $200,000 to $400,000
4th bracket: 1305 records which is the group of customer who earned income more than $400,000


In [16]:
full_data_merge_age_risk['AMT_INCOME_TOTAL'] = np.where(full_data_merge_age_risk['AMT_INCOME_TOTAL'] < 100000,1,\
         np.where((full_data_merge_age_risk['AMT_INCOME_TOTAL'] >= 100000)& \
                  (full_data_merge_age_risk['AMT_INCOME_TOTAL'] < 200000),2,\
                  np.where((full_data_merge_age_risk['AMT_INCOME_TOTAL'] >= 200000)& \
                  (full_data_merge_age_risk['AMT_INCOME_TOTAL'] < 400000),3,4)))

In [17]:
full_data_merge_age_risk['AMT_INCOME_TOTAL'].value_counts()

2    17897
3    12169
1     5086
4     1305
Name: AMT_INCOME_TOTAL, dtype: int64

In [18]:
value_cnt_norm(full_data_merge_age_risk,'AMT_INCOME_TOTAL')

,AMT_INCOME_TOTAL_CNT,FREQUENCY
2,17897,49.090710
3,12169,33.379049
1,5086,13.950682
4,1305,3.579559


In [19]:
#Birthdate

In [20]:
# day employ

# convert value of these columns to categorical 


CODE_GENDER_CNT  FREQUENCY
 F            24430  67.010451
 M            12027  32.989549,
    FLAG_OWN_CAR_CNT  FREQUENCY
 N             22614   62.02924
 Y             13843   37.97076,
    FLAG_OWN_REALTY_CNT  FREQUENCY
 Y                24506  67.218915
 N                11951  32.7

In [21]:
full_data_merge_age_risk['CODE_GENDER'] = full_data_merge_age_risk['CODE_GENDER'].apply(lambda x: 1 if x == 'F' else 0)
full_data_merge_age_risk['FLAG_OWN_CAR'] = full_data_merge_age_risk['FLAG_OWN_CAR'].apply(lambda x: 1 if x == 'Y' else 0)

full_data_merge_age_risk['FLAG_OWN_REALTY'] = full_data_merge_age_risk['FLAG_OWN_REALTY'].apply(lambda x: 1 if x == 'Y' else 0)


### Change NAME_EDUCATION_TYPE	object	to ordinal


In [22]:
print(full_data_merge_age_risk['NAME_EDUCATION_TYPE'].value_counts())

Secondary / secondary special    24777
Higher education                  9864
Incomplete higher                 1410
Lower secondary                    374
Academic degree                     32
Name: NAME_EDUCATION_TYPE, dtype: int64


In [23]:
full_data_merge_age_risk['NAME_EDUCATION_TYPE'] = np.where(full_data_merge_age_risk['NAME_EDUCATION_TYPE'] == "Lower secondary",0,\
        np.where(full_data_merge_age_risk['NAME_EDUCATION_TYPE'] == "Secondary / secondary special",1,\
                np.where(full_data_merge_age_risk['NAME_EDUCATION_TYPE'] == "Incomplete higher",2,4)))

print(full_data_merge_age_risk['NAME_EDUCATION_TYPE'].value_counts())

1    24777
4     9896
2     1410
0      374
Name: NAME_EDUCATION_TYPE, dtype: int64


NAME_INCOME_TYPE	object	36457	5	0	0.00


In [30]:
print(full_data_merge_age_risk['NAME_INCOME_TYPE'].value_counts())

Working                 18819
Commercial associate     8490
Pensioner                6152
State servant            2985
Student                    11
Name: NAME_INCOME_TYPE, dtype: int64


8	NAME_FAMILY_STATUS	object	36457	5	0	0.00


In [31]:
print(full_data_merge_age_risk['NAME_FAMILY_STATUS'].value_counts())

Married                 25048
Single / not married     4829
Civil marriage           2945
Separated                2103
Widow                    1532
Name: NAME_FAMILY_STATUS, dtype: int64


9	NAME_HOUSING_TYPE

In [32]:
print(full_data_merge_age_risk['NAME_HOUSING_TYPE'].value_counts())

House / apartment      32548
With parents            1776
Municipal apartment     1128
Rented apartment         575
Office apartment         262
Co-op apartment          168
Name: NAME_HOUSING_TYPE, dtype: int64



# check if there are any missing values

In [24]:
def show_missing(df):
    """Return a Pandas dataframe describing the contents of a source dataframe including missing values."""
    
    variables = []
    dtypes = []
    count = []
    unique = []
    missing = []
    pc_missing = []
    
    for item in df.columns:
        variables.append(item)
        dtypes.append(df[item].dtype)
        count.append(len(df[item]))
        unique.append(len(df[item].unique()))
        missing.append(df[item].isna().sum())
        pc_missing.append(round((df[item].isna().sum() / len(df[item])) * 100, 2))

    output = pd.DataFrame({
        'variable': variables, 
        'dtype': dtypes,
        'count': count,
        'unique': unique,
        'missing': missing, 
        'pc_missing': pc_missing
    })    
        
    return output

show_missing(full_data_merge_age_risk)

,variable,dtype,count,unique,missing,pc_missing
0,ID,int64,36457,36457,0,0.00
1,CODE_GENDER,int64,36457,2,0,0.00
2,FLAG_OWN_CAR,int64,36457,2,0,0.00
3,FLAG_OWN_REALTY,int64,36457,2,0,0.00
4,CNT_CHILDREN,int64,36457,9,0,0.00
5,AMT_INCOME_TOTAL,int64,36457,4,0,0.00
6,NAME_INCOME_TYPE,object,36457,5,0,0.00
7,NAME_EDUCATION_TYPE,int64,36457,4,0,0.00
8,NAME_FAMILY_STATUS,object,36457,5,0,0.00
9,NAME_HOUSING_TYPE,object,36457,6,0,0.00


In [25]:
full_data_merge_age_risk.OCCUPATION_TYPE.value_counts()


Laborers                 6211
Core staff               3591
Sales staff              3485
Managers                 3012
Drivers                  2138
High skill tech staff    1383
Accountants              1241
Medicine staff           1207
Cooking staff             655
Security staff            592
Cleaning staff            551
Private service staff     344
Low-skill Laborers        175
Waiters/barmen staff      174
Secretaries               151
HR staff                   85
Realty agents              79
IT staff                   60
Name: OCCUPATION_TYPE, dtype: int64

In [26]:
# since OCCUPATION_TYPE miss value contains 31% of the dataframe
# I decide to drop this column
drop_col = ['OCCUPATION_TYPE']
df2 =full_data_merge_age_risk.drop(columns = drop_col)
df2.shape

(36457, 19)

# Split train and test data

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df2.iloc[:,:-1],\
                                                    df2[['HIGH_RISK']],\
                                                    test_size = 0.2, random_state = 42)

In [29]:
train_df = pd.concat([X_train,y_train], axis = 1)
test_df = pd.concat([X_test,y_test], axis = 1)
train_df.to_csv('dataset/cc_train.csv', index = False)
test_df.to_csv('dataset/cc_test.csv', index = False)
print('\033[94m')
print('*'*60)
print(f'There are {train_df.shape[1]} columns and {train_df.shape[0]} rows in Train set')
print(f'There are {test_df.shape[1]} columns and {test_df.shape[0]} rows in Test set')


************************************************************
There are 19 columns and 29165 rows in Train set
There are 19 columns and 7292 rows in Test set
